In [4]:
from ultralytics import YOLO
import numpy as np

In [6]:
model = YOLO('yolo11s.pt')

results = model("detecton_box_test/0a0a0b1a-7c39d841.jpg")


metrics = model.val(data='bdd100k.yaml', save_json=True)

print(f"Mean Average Precision (mAP50-95): {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

class_names = model.names
for i, score in enumerate(metrics.box.maps):
    print(f"Class {class_names[i]}: {score:.4f}")


image 1/1 /Users/beaulin/DS 340 Project/Analysis/road_object_detec/detecton_box_test/0a0a0b1a-7c39d841.jpg: 384x640 2 cars, 3 trucks, 57.9ms
Speed: 1.5ms preprocess, 57.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
Ultralytics 8.3.230 🚀 Python-3.12.2 torch-2.9.1 CPU (Apple M2 Pro)
image 1/1 /Users/beaulin/DS 340 Project/Analysis/road_object_detec/detecton_box_test/0a0a0b1a-7c39d841.jpg: 384x640 2 cars, 3 trucks, 57.9ms
Speed: 1.5ms preprocess, 57.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
Ultralytics 8.3.230 🚀 Python-3.12.2 torch-2.9.1 CPU (Apple M2 Pro)
val: Fast image access ✅ (ping: 0.1±0.1 ms, read: 300.1±81.7 MB/s, size: 62.7 KB)
val: Fast image access ✅ (ping: 0.1±0.1 ms, read: 300.1±81.7 MB/s, size: 62.7 KB)
val: Scanning /Users/beaulin/DS 340 Project/Analysis/road_object_detec/data/labels/val... 10000 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 10000/10000 4.4Kit/s 2.3s0s
val: Scanning /Users/beaulin/DS 340 Project/Analy